# Tentative report: The purpose of this file is to test and check the phenomenon of how a data set's learnability by a neural network varies based on its complexity.

Current Industry Standard is to use the following formula for evaluating complexity:

## Quantity 1:
\begin{align*}
Q_1 &= \frac{(\frac{1}{N}*\sum\limits_{i=1}^N{(f(i)-\mu)^2})^\text{1/2}}{\mu}
\end{align*}

Notice how for simple line though, this may be large, so we also use the following quantity:

## Quantity 2:
\begin{align*}
    Q_2 &= \frac{\sqrt{\frac{1}{N-1}*\sum\limits_{i=1}^N{(f(i)-f(i-1)-\mu_\Delta)^2}}}{\mu_\Delta}\\
\end{align*}




Where

\begin{align*}
    \mu_\Delta &= \frac{1}{N-1}*\sum\limits_{i=2}^N{f(i)-f(i-1)} &=\frac{f(N)-f(1)}{N-1}
\end{align*}




If either of these values are "small" we deem the set learnable, lets investigate the relationship between different values of these two quantities and the loss during learning.

We first normalize the data (we tentatively do a linear map of our data set from the original domain [0,800] to [0,1]) to feed it into a neural network

The function_info(function) will run the neural network from dataset generated by a built-in function, or importing a list (check function_array[] section). Each function has a training set of 800 points and a test set of 200 points by default, This can be adjusted.

The random_fun () will run the neural network from a random dataset with y-value ranging from [-2,2] as the current built-in function's y-value also ranging from [-2.2] after mapping.

The current findings are:

The basic approach meets our expectation: the linear data has a complexity very close to zero 1.1046610265648073e-14 and very small loss. On the other hand the random data has a relative large complexity with a very large loss, which is greater than 1. Simply not learnable and you can also see that during training, the neural network never converges.

There are functions that seems learnable, (we tentatively assume that learnability is related to the loss during training and the loss during testing, if both losses are small, then the neural network generates a good estimate function) with a complexity greater than the one from random case. e.g "f = lambda x, i=i : (np.sin(1000/x))((1/10)x**(1+(1/i))) if x!=0 else 0" the loss from test is still very small when complexity goes to 120


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from sklearn import metrics
import numpy as np
import random

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import backend
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
import math
from math import log
import numpy as np
from numpy.linalg import norm
import matplotlib
import matplotlib.pyplot as plt

Define your function to be learned:

Normalization of function using linear map:

In [ ]:
def LinearMap(old_interval, new_interval, function_val):
      new_dist = new_interval[1]-new_interval[0]
      old_dist = old_interval[1]-old_interval[0]
      linear_map = (function_val - old_interval[0]) * new_dist / old_dist + new_interval[0]

      return linear_map

def LinearMapList(old_interval,new_interval,list):
  new_list = []
  for i in range(len(list)):
    new_list.append(LinearMap(old_interval, new_interval, list[i]))

  return new_list


LinearMapList([1,5],[0,1],[2,3,4,5,6])

[0.25, 0.5, 0.75, 1.0, 1.25]

Quantity 1 Calculation:
\begin{align*}
Q_1 &= \frac{(\frac{1}{N}*\sum\limits_{i=1}^N{(f(i)-\mu)^2})^\text{1/2}}{\mu}
\end{align*}



In [ ]:
def function1(outputf):
  outputf=np.array(outputf)
  mean=np.mean(outputf)
  mse=np.sum(np.square(outputf-mean))/len(outputf)
  return mse**(0.5)/mean

Quantity 2 Calculation:
\begin{align*}
    Q_2 &= \frac{\sqrt{\frac{1}{N-1}*\sum\limits_{i=1}^N{(f(i)-f(i-1)-\mu_\Delta)^2}}}{\mu_\Delta}\\
\end{align*}




Where

\begin{align*}
    \mu_\Delta &= \frac{1}{N-1}*\sum\limits_{i=2}^N{f(i)-f(i-1)} &=\frac{f(N)-f(1)}{N-1}
\end{align*}



In [ ]:
def function2(outputf):
  outputf=np.array(outputf)
  deltaf=[]

  for i in range(1,len(outputf)):
    deltaf.append(outputf[i]-outputf[i-1])

  #mean=np.mean(deltaf)
  delmean = (outputf[-1]-outputf[0])/(len(outputf)-1)

  mse=np.sum(np.square(deltaf-delmean))/len(deltaf)
  return mse**(0.5)/delmean


It may be useful to have a function that returns the min of these two values:

In [ ]:
def complexity(val1,val2):
  if (abs(val1)>=abs(val2)):
    return abs(val2)

  else: return abs(val1)

generate train/test x and train/test y, custome the list yourself

In [ ]:
train_x = []

for i in range(0,800):   #train size: 800
  train_x.append(i/1000)

test_x = []

for i in range(800,1001):  #test size: 201
  test_x.append(i/1000)

train_y = []

test_y = []

First we test our data in random case

In [ ]:
def random_floats(low, high, size):
   return  [random.uniform(low,high) for _ in range(size)]

def random_fun (train_x, test_x, train_y, test_y, seed):

  tensorflow.random.set_seed(seed)
  model2 = Sequential()
  model2.add(Dense(100, input_dim=1, activation='relu'))
  model2.add(Dense(10, activation='relu'))
  model2.add(Dense(1))
  model2.compile(loss='mean_squared_error', optimizer='adam')

  model2.fit(train_x,train_y,verbose=2,epochs=20)

  #processes the data for the model

  pred1 = model2.predict(test_x)


  score1 = np.sqrt(metrics.mean_squared_error(pred1, test_y))

  sum1 = function1(test_y)

  sum2 = function2(test_y)

  fun_complexity = complexity(sum1, sum2)

  print("Quantity1: " ,sum1, "\nQuantity2: ", sum2)


  return score1, fun_complexity

Test random fun:

In [ ]:
train_x_list = train_x
test_x_list = test_x

train_y_list = random_floats(-2, 2, 800)
test_y_list = random_floats(-2, 2, 201)

random_score, random_complexity = random_fun(train_x_list, test_x_list, train_y_list, test_y_list, 3)

print("loss for this random data: ", random_score)
print("complexity:",random_complexity)

Epoch 1/20
25/25 - 1s - loss: 1.3046 - 665ms/epoch - 27ms/step
Epoch 2/20
25/25 - 0s - loss: 1.3017 - 32ms/epoch - 1ms/step
Epoch 3/20
25/25 - 0s - loss: 1.3014 - 33ms/epoch - 1ms/step
Epoch 4/20
25/25 - 0s - loss: 1.3012 - 29ms/epoch - 1ms/step
Epoch 5/20
25/25 - 0s - loss: 1.3010 - 29ms/epoch - 1ms/step
Epoch 6/20
25/25 - 0s - loss: 1.3009 - 29ms/epoch - 1ms/step
Epoch 7/20
25/25 - 0s - loss: 1.3007 - 29ms/epoch - 1ms/step
Epoch 8/20
25/25 - 0s - loss: 1.3006 - 29ms/epoch - 1ms/step
Epoch 9/20
25/25 - 0s - loss: 1.3004 - 41ms/epoch - 2ms/step
Epoch 10/20
25/25 - 0s - loss: 1.3004 - 30ms/epoch - 1ms/step
Epoch 11/20
25/25 - 0s - loss: 1.3002 - 29ms/epoch - 1ms/step
Epoch 12/20
25/25 - 0s - loss: 1.3002 - 29ms/epoch - 1ms/step
Epoch 13/20
25/25 - 0s - loss: 1.3001 - 28ms/epoch - 1ms/step
Epoch 14/20
25/25 - 0s - loss: 1.3000 - 29ms/epoch - 1ms/step
Epoch 15/20
25/25 - 0s - loss: 1.2999 - 30ms/epoch - 1ms/step
Epoch 16/20
25/25 - 0s - loss: 1.2999 - 31ms/epoch - 1ms/step
Epoch 17/20
25/

Neural Network performance test for any function list other than random:

In [ ]:
seed = 0 # try more different values.. or no value

def function_any (train_x, test_x, train_y, test_y, num_of_epoch):

  #model for the data
  tensorflow.random.set_seed(seed)
  model1 = Sequential()
  model1.add(Dense(100, input_dim=1, activation='relu')) # Hidden 1 #simoid, relu, tanh
  model1.add(Dense(10, activation='relu')) # Hidden 2
  model1.add(Dense(1)) # Output
  model1.compile(loss='mean_squared_error', optimizer='adam')

  model1.fit(train_x,train_y,verbose=2,epochs=num_of_epoch)


  pred1 = model1.predict(test_x)


  score1 = np.sqrt(metrics.mean_squared_error(pred1, test_y))

  #pvariance1 = (test_y.std()-score1)/test_y.std() # for later use

  sum1 = function1(train_y)

  print("Quantity 1: ",sum1)

  sum2 = function2(train_y)

  print("Quantity 2: ",sum2)

  fun_complexity = complexity(sum1, sum2)

  print("Loss:", score1)

  return score1, fun_complexity

Demo, test our function with a straight line, the main purpose of this demo is to show the comparison between quantity 1 and quantity 2

In [ ]:
train_x_line = train_x
test_x_line = test_x

train_y_line = [0.0, 0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.06, 0.062, 0.064, 0.066, 0.068, 0.07, 0.072, 0.074, 0.076, 0.078, 0.08, 0.082, 0.084, 0.086, 0.088, 0.09, 0.092, 0.094, 0.096, 0.098, 0.1, 0.102, 0.104, 0.106, 0.108, 0.11, 0.112, 0.114, 0.116, 0.118, 0.12, 0.122, 0.124, 0.126, 0.128, 0.13, 0.132, 0.134, 0.136, 0.138, 0.14, 0.142, 0.144, 0.146, 0.148, 0.15, 0.152, 0.154, 0.156, 0.158, 0.16, 0.162, 0.164, 0.166, 0.168, 0.17, 0.172, 0.174, 0.176, 0.178, 0.18, 0.182, 0.184, 0.186, 0.188, 0.19, 0.192, 0.194, 0.196, 0.198, 0.2, 0.202, 0.204, 0.206, 0.208, 0.21, 0.212, 0.214, 0.216, 0.218, 0.22, 0.222, 0.224, 0.226, 0.228, 0.23, 0.232, 0.234, 0.236, 0.238, 0.24, 0.242, 0.244, 0.246, 0.248, 0.25, 0.252, 0.254, 0.256, 0.258, 0.26, 0.262, 0.264, 0.266, 0.268, 0.27, 0.272, 0.274, 0.276, 0.278, 0.28, 0.282, 0.284, 0.286, 0.288, 0.29, 0.292, 0.294, 0.296, 0.298, 0.3, 0.302, 0.304, 0.306, 0.308, 0.31, 0.312, 0.314, 0.316, 0.318, 0.32, 0.322, 0.324, 0.326, 0.328, 0.33, 0.332, 0.334, 0.336, 0.338, 0.34, 0.342, 0.344, 0.346, 0.348, 0.35, 0.352, 0.354, 0.356, 0.358, 0.36, 0.362, 0.364, 0.366, 0.368, 0.37, 0.372, 0.374, 0.376, 0.378, 0.38, 0.382, 0.384, 0.386, 0.388, 0.39, 0.392, 0.394, 0.396, 0.398, 0.4, 0.402, 0.404, 0.406, 0.408, 0.41, 0.412, 0.414, 0.416, 0.418, 0.42, 0.422, 0.424, 0.426, 0.428, 0.43, 0.432, 0.434, 0.436, 0.438, 0.44, 0.442, 0.444, 0.446, 0.448, 0.45, 0.452, 0.454, 0.456, 0.458, 0.46, 0.462, 0.464, 0.466, 0.468, 0.47, 0.472, 0.474, 0.476, 0.478, 0.48, 0.482, 0.484, 0.486, 0.488, 0.49, 0.492, 0.494, 0.496, 0.498, 0.5, 0.502, 0.504, 0.506, 0.508, 0.51, 0.512, 0.514, 0.516, 0.518, 0.52, 0.522, 0.524, 0.526, 0.528, 0.53, 0.532, 0.534, 0.536, 0.538, 0.54, 0.542, 0.544, 0.546, 0.548, 0.55, 0.552, 0.554, 0.556, 0.558, 0.56, 0.562, 0.564, 0.566, 0.568, 0.57, 0.572, 0.574, 0.576, 0.578, 0.58, 0.582, 0.584, 0.586, 0.588, 0.59, 0.592, 0.594, 0.596, 0.598, 0.6, 0.602, 0.604, 0.606, 0.608, 0.61, 0.612, 0.614, 0.616, 0.618, 0.62, 0.622, 0.624, 0.626, 0.628, 0.63, 0.632, 0.634, 0.636, 0.638, 0.64, 0.642, 0.644, 0.646, 0.648, 0.65, 0.652, 0.654, 0.656, 0.658, 0.66, 0.662, 0.664, 0.666, 0.668, 0.67, 0.672, 0.674, 0.676, 0.678, 0.68, 0.682, 0.684, 0.686, 0.688, 0.69, 0.692, 0.694, 0.696, 0.698, 0.7, 0.702, 0.704, 0.706, 0.708, 0.71, 0.712, 0.714, 0.716, 0.718, 0.72, 0.722, 0.724, 0.726, 0.728, 0.73, 0.732, 0.734, 0.736, 0.738, 0.74, 0.742, 0.744, 0.746, 0.748, 0.75, 0.752, 0.754, 0.756, 0.758, 0.76, 0.762, 0.764, 0.766, 0.768, 0.77, 0.772, 0.774, 0.776, 0.778, 0.78, 0.782, 0.784, 0.786, 0.788, 0.79, 0.792, 0.794, 0.796, 0.798, 0.8, 0.802, 0.804, 0.806, 0.808, 0.81, 0.812, 0.814, 0.816, 0.818, 0.82, 0.822, 0.824, 0.826, 0.828, 0.83, 0.832, 0.834, 0.836, 0.838, 0.84, 0.842, 0.844, 0.846, 0.848, 0.85, 0.852, 0.854, 0.856, 0.858, 0.86, 0.862, 0.864, 0.866, 0.868, 0.87, 0.872, 0.874, 0.876, 0.878, 0.88, 0.882, 0.884, 0.886, 0.888, 0.89, 0.892, 0.894, 0.896, 0.898, 0.9, 0.902, 0.904, 0.906, 0.908, 0.91, 0.912, 0.914, 0.916, 0.918, 0.92, 0.922, 0.924, 0.926, 0.928, 0.93, 0.932, 0.934, 0.936, 0.938, 0.94, 0.942, 0.944, 0.946, 0.948, 0.95, 0.952, 0.954, 0.956, 0.958, 0.96, 0.962, 0.964, 0.966, 0.968, 0.97, 0.972, 0.974, 0.976, 0.978, 0.98, 0.982, 0.984, 0.986, 0.988, 0.99, 0.992, 0.994, 0.996, 0.998, 1.0, 1.002, 1.004, 1.006, 1.008, 1.01, 1.012, 1.014, 1.016, 1.018, 1.02, 1.022, 1.024, 1.026, 1.028, 1.03, 1.032, 1.034, 1.036, 1.038, 1.04, 1.042, 1.044, 1.046, 1.048, 1.05, 1.052, 1.054, 1.056, 1.058, 1.06, 1.062, 1.064, 1.066, 1.068, 1.07, 1.072, 1.074, 1.076, 1.078, 1.08, 1.082, 1.084, 1.086, 1.088, 1.09, 1.092, 1.094, 1.096, 1.098, 1.1, 1.102, 1.104, 1.106, 1.108, 1.11, 1.112, 1.114, 1.116, 1.118, 1.12, 1.122, 1.124, 1.126, 1.128, 1.13, 1.132, 1.134, 1.136, 1.138, 1.14, 1.142, 1.144, 1.146, 1.148, 1.15, 1.152, 1.154, 1.156, 1.158, 1.16, 1.162, 1.164, 1.166, 1.168, 1.17, 1.172, 1.174, 1.176, 1.178, 1.18, 1.182, 1.184, 1.186, 1.188, 1.19, 1.192, 1.194, 1.196, 1.198, 1.2, 1.202, 1.204, 1.206, 1.208, 1.21, 1.212, 1.214, 1.216, 1.218, 1.22, 1.222, 1.224, 1.226, 1.228, 1.23, 1.232, 1.234, 1.236, 1.238, 1.24, 1.242, 1.244, 1.246, 1.248, 1.25, 1.252, 1.254, 1.256, 1.258, 1.26, 1.262, 1.264, 1.266, 1.268, 1.27, 1.272, 1.274, 1.276, 1.278, 1.28, 1.282, 1.284, 1.286, 1.288, 1.29, 1.292, 1.294, 1.296, 1.298, 1.3, 1.302, 1.304, 1.306, 1.308, 1.31, 1.312, 1.314, 1.316, 1.318, 1.32, 1.322, 1.324, 1.326, 1.328, 1.33, 1.332, 1.334, 1.336, 1.338, 1.34, 1.342, 1.344, 1.346, 1.348, 1.35, 1.352, 1.354, 1.356, 1.358, 1.36, 1.362, 1.364, 1.366, 1.368, 1.37, 1.372, 1.374, 1.376, 1.378, 1.38, 1.382, 1.384, 1.386, 1.388, 1.39, 1.392, 1.394, 1.396, 1.398, 1.4, 1.402, 1.404, 1.406, 1.408, 1.41, 1.412, 1.414, 1.416, 1.418, 1.42, 1.422, 1.424, 1.426, 1.428, 1.43, 1.432, 1.434, 1.436, 1.438, 1.44, 1.442, 1.444, 1.446, 1.448, 1.45, 1.452, 1.454, 1.456, 1.458, 1.46, 1.462, 1.464, 1.466, 1.468, 1.47, 1.472, 1.474, 1.476, 1.478, 1.48, 1.482, 1.484, 1.486, 1.488, 1.49, 1.492, 1.494, 1.496, 1.498, 1.5, 1.502, 1.504, 1.506, 1.508, 1.51, 1.512, 1.514, 1.516, 1.518, 1.52, 1.522, 1.524, 1.526, 1.528, 1.53, 1.532, 1.534, 1.536, 1.538, 1.54, 1.542, 1.544, 1.546, 1.548, 1.55, 1.552, 1.554, 1.556, 1.558, 1.56, 1.562, 1.564, 1.566, 1.568, 1.57, 1.572, 1.574, 1.576, 1.578, 1.58, 1.582, 1.584, 1.586, 1.588, 1.59, 1.592, 1.594, 1.596, 1.598]
test_y_line = [1.6, 1.602, 1.604, 1.606, 1.608, 1.61, 1.612, 1.614, 1.616, 1.618, 1.62, 1.622, 1.624, 1.626, 1.628, 1.63, 1.632, 1.634, 1.636, 1.638, 1.64, 1.642, 1.644, 1.646, 1.648, 1.65, 1.652, 1.654, 1.656, 1.658, 1.66, 1.662, 1.664, 1.666, 1.668, 1.67, 1.672, 1.674, 1.676, 1.678, 1.68, 1.682, 1.684, 1.686, 1.688, 1.69, 1.692, 1.694, 1.696, 1.698, 1.7, 1.702, 1.704, 1.706, 1.708, 1.71, 1.712, 1.714, 1.716, 1.718, 1.72, 1.722, 1.724, 1.726, 1.728, 1.73, 1.732, 1.734, 1.736, 1.738, 1.74, 1.742, 1.744, 1.746, 1.748, 1.75, 1.752, 1.754, 1.756, 1.758, 1.76, 1.762, 1.764, 1.766, 1.768, 1.77, 1.772, 1.774, 1.776, 1.778, 1.78, 1.782, 1.784, 1.786, 1.788, 1.79, 1.792, 1.794, 1.796, 1.798, 1.8, 1.802, 1.804, 1.806, 1.808, 1.81, 1.812, 1.814, 1.816, 1.818, 1.82, 1.822, 1.824, 1.826, 1.828, 1.83, 1.832, 1.834, 1.836, 1.838, 1.84, 1.842, 1.844, 1.846, 1.848, 1.85, 1.852, 1.854, 1.856, 1.858, 1.86, 1.862, 1.864, 1.866, 1.868, 1.87, 1.872, 1.874, 1.876, 1.878, 1.88, 1.882, 1.884, 1.886, 1.888, 1.89, 1.892, 1.894, 1.896, 1.898, 1.9, 1.902, 1.904, 1.906, 1.908, 1.91, 1.912, 1.914, 1.916, 1.918, 1.92, 1.922, 1.924, 1.926, 1.928, 1.93, 1.932, 1.934, 1.936, 1.938, 1.94, 1.942, 1.944, 1.946, 1.948, 1.95, 1.952, 1.954, 1.956, 1.958, 1.96, 1.962, 1.964, 1.966, 1.968, 1.97, 1.972, 1.974, 1.976, 1.978, 1.98, 1.982, 1.984, 1.986, 1.988, 1.99, 1.992, 1.994, 1.996, 1.998, 2.0]

#well, that is just a line

In [ ]:
line_score, line_complexity = function_any(train_x_line, test_x_line, train_y_line, test_y_line, 20)

print("loss for this random data: ", line_score)
print("complexity:",line_complexity)

Epoch 1/20
25/25 - 1s - loss: 0.8341 - 501ms/epoch - 20ms/step
Epoch 2/20
25/25 - 0s - loss: 0.6028 - 28ms/epoch - 1ms/step
Epoch 3/20
25/25 - 0s - loss: 0.4120 - 28ms/epoch - 1ms/step
Epoch 4/20
25/25 - 0s - loss: 0.2425 - 28ms/epoch - 1ms/step
Epoch 5/20
25/25 - 0s - loss: 0.1254 - 28ms/epoch - 1ms/step
Epoch 6/20
25/25 - 0s - loss: 0.0705 - 27ms/epoch - 1ms/step
Epoch 7/20
25/25 - 0s - loss: 0.0492 - 40ms/epoch - 2ms/step
Epoch 8/20
25/25 - 0s - loss: 0.0343 - 29ms/epoch - 1ms/step
Epoch 9/20
25/25 - 0s - loss: 0.0216 - 32ms/epoch - 1ms/step
Epoch 10/20
25/25 - 0s - loss: 0.0118 - 32ms/epoch - 1ms/step
Epoch 11/20
25/25 - 0s - loss: 0.0055 - 29ms/epoch - 1ms/step
Epoch 12/20
25/25 - 0s - loss: 0.0022 - 28ms/epoch - 1ms/step
Epoch 13/20
25/25 - 0s - loss: 8.5473e-04 - 42ms/epoch - 2ms/step
Epoch 14/20
25/25 - 0s - loss: 3.8807e-04 - 29ms/epoch - 1ms/step
Epoch 15/20
25/25 - 0s - loss: 2.2821e-04 - 32ms/epoch - 1ms/step
Epoch 16/20
25/25 - 0s - loss: 1.5786e-04 - 31ms/epoch - 1ms/step

Useful tool: Linear Map function , which will map data from [a,b] to [c,d], by the function $y = c + \frac{d-c}{b-a}*(x-a)$

In [ ]:
def LinearMap(old_interval, new_interval, function_val):
      new_dist = new_interval[1]-new_interval[0]
      old_dist = old_interval[1]-old_interval[0]
      linear_map = (function_val - old_interval[0]) * new_dist / old_dist + new_interval[0]

      return linear_map

def LinearMapList(old_interval,new_interval,list):
  new_list = []
  for i in range(len(list)):
    new_list.append(LinearMap(old_interval, new_interval, list[i]))

  return new_list


LinearMapList([1,5],[0,1],[2,3,4,5,6])

[0.25, 0.5, 0.75, 1.0, 1.25]

In [ ]:
g = lambda x: 2*x

x_before_map = []

for i in range(0,800):
  x_before_map.append(i)

def function_list(f):
  series = []
  for i in x_before_map:
    series.append(f(i))
  return series

train_y_before_map = function_list(g)

print(train_y_before_map)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370, 372, 374, 376, 378, 380, 382, 384, 386, 388, 390, 392, 394, 396, 398, 400, 402, 404, 406, 408, 410, 412, 414, 416, 418, 420,

Then do a linear map of function g from [0,800] to [0,0.8], compare result with "train_y_line" above

In [ ]:
train_y_after_map = LinearMapList([0,800],[0,800/1000],function_list(g))

print(train_y_after_map)

[0.0, 0.002, 0.004, 0.006000000000000001, 0.008, 0.01, 0.012000000000000002, 0.014000000000000002, 0.016, 0.018000000000000002, 0.02, 0.022000000000000002, 0.024000000000000004, 0.026000000000000002, 0.028000000000000004, 0.03, 0.032, 0.034, 0.036000000000000004, 0.038000000000000006, 0.04, 0.042, 0.044000000000000004, 0.046000000000000006, 0.04800000000000001, 0.05, 0.052000000000000005, 0.054000000000000006, 0.05600000000000001, 0.05800000000000001, 0.06, 0.062, 0.064, 0.066, 0.068, 0.07, 0.07200000000000001, 0.07400000000000001, 0.07600000000000001, 0.07800000000000001, 0.08, 0.08200000000000002, 0.084, 0.086, 0.08800000000000001, 0.09, 0.09200000000000001, 0.094, 0.09600000000000002, 0.098, 0.1, 0.10200000000000001, 0.10400000000000001, 0.10600000000000001, 0.10800000000000001, 0.11, 0.11200000000000002, 0.114, 0.11600000000000002, 0.11800000000000001, 0.12, 0.12200000000000001, 0.124, 0.126, 0.128, 0.13, 0.132, 0.134, 0.136, 0.138, 0.14, 0.14200000000000002, 0.14400000000000002, 0